In [ ]:
# Import necessary libraries
import torch # PyTorch library
import torch.nn as nn # Neural network module
import torch.optim as optim # Optimization algorithms
import torch.nn.functional as F # Functional interface
import torchvision # Computer vision library
import torchvision.transforms as transforms # Image transformations
import matplotlib.pyplot as plt # Plotting library

In [ ]:
# Define transformations for the training and testing data sets and load CIFAR-10 dataset from torchvision datasets module with defined transformations
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform) # Download and load training data
trainloader = torch.utils.data.DataLoader(trainset, batch_size=4, shuffle=True, num_workers=2) # Create data loader for training data

testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform) # Download and load testing data
testloader = torch.utils.data.DataLoader(testset, batch_size=4, shuffle=False, num_workers=2) # Create data loader for testing data

100%|██████████| 170M/170M [00:06<00:00, 25.6MB/s] 


In [ ]:
# Define a simple Convolutional Neural Network (CNN) model using nn.Module class from torch.nn module 
class Net(nn.Module): # Define a simple CNN model
    def __init__(self): # Initialize the model
        super(Net, self).__init__() # Call the parent class constructor
        self.conv1 = nn.Conv2d(3, 6, 5) # First convolutional layer
        self.pool = nn.MaxPool2d(2, 2) # Max pooling layer
        self.conv2 = nn.Conv2d(6, 16, 5) # Second convolutional layer
        self.fc1 = nn.Linear(16 * 5 * 5, 120) # First fully connected layer
        self.fc2 = nn.Linear(120, 84) # Second fully connected layer
        self.fc3 = nn.Linear(84, 10) # Third fully connected layer

    def forward(self, x): # Define the forward pass
        x = self.pool(F.relu(self.conv1(x))) # Apply first convolutional layer, ReLU activation, and max pooling
        x = self.pool(F.relu(self.conv2(x))) # Apply second convolutional layer, ReLU activation, and max pooling
        x = x.view(-1, 16 * 5 * 5) # Flatten the tensor
        x = F.relu(self.fc1(x)) # Apply first fully connected layer and ReLU activation
        x = F.relu(self.fc2(x)) # Apply second fully connected layer and ReLU activation
        x = self.fc3(x) # Apply third fully connected layer
        return x # Return the output

net = Net() # Instantiate the model 

In [4]:
# Define a loss function and an optimizer for training the model 
criterion = nn.CrossEntropyLoss() # Cross-entropy loss function
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9) # Stochastic Gradient Descent (SGD) optimizer with learning rate and momentum


In [5]:
# Train the model using the training data loader 
for epoch in range(2):  # loop over the dataset multiple times
    running_loss = 0.0 # Initialize running loss
    for i, data in enumerate(trainloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data # Get inputs and labels from data loader

        # zero the parameter gradients
        optimizer.zero_grad() 

        # forward + backward + optimize
        outputs = net(inputs) # Forward pass
        loss = criterion(outputs, labels) # Compute loss
        loss.backward() # Backward pass
        optimizer.step() # Update weights

        # print statistics
        running_loss += loss.item() # Accumulate loss
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 2000:.3f}') # Print average loss
            running_loss = 0.0 # Reset running loss

print('Finished Training') 

[1,  2000] loss: 2.209
[1,  4000] loss: 1.875
[1,  6000] loss: 1.672
[1,  8000] loss: 1.610
[1, 10000] loss: 1.545
[1, 12000] loss: 1.477
[2,  2000] loss: 1.413
[2,  4000] loss: 1.386
[2,  6000] loss: 1.368
[2,  8000] loss: 1.316
[2, 10000] loss: 1.317
[2, 12000] loss: 1.292
Finished Training


In [6]:
# Evaluate the model using the test data loader and calculate accuracy 
correct = 0 # Initialize correct predictions count
total = 0 # Initialize total predictions count
with torch.no_grad(): # Disable gradient calculation for evaluation
    for data in testloader: # Iterate over test data
        images, labels = data # Get images and labels
        outputs = net(images) # Get model outputs
        _, predicted = torch.max(outputs.data, 1) # Get predicted class
        total += labels.size(0) # Update total count
        correct += (predicted == labels).sum().item() # Update correct count

print(f'Accuracy of the network on the 10000 test images: {100 * correct / total} %') # Print accuracy 

Accuracy of the network on the 10000 test images: 54.64 %


In [7]:
# Save the trained model to a file using torch.save() function
PATH = './cifar_net.pth' # Define the file path to save the model
torch.save(net.state_dict(), PATH) # Save the model state dictionary to the file

In [8]:
# Load the saved model from the file using torch.load() function 
net = Net() # Instantiate the model
net.load_state_dict(torch.load(PATH)) # Load the model state dictionary from the file

<All keys matched successfully>